In [366]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [367]:
df = pd.read_csv('../data/상장raw2.csv', encoding='euc-kr')
df상장 = pd.read_excel('../data/kospi+kodaq_dp(등급).xls')

In [368]:
df['회계년도']= df['회계년도'].replace('/','').astype(str).str[:4].astype(int)
df = df[df['회계년도'] == 2022]

In [369]:
df['총자산대비영업현금흐름'] = df['영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)'] / df['자산(*)(IFRS연결)(천원)']
df['PBR'] = df['PBR(최고)(IFRS)'] + df['PBR(최저)(IFRS)'] /2
df['자산(*)(IFRS연결)(천원)'] = df['자산(*)(IFRS연결)(천원)'] * 1000
df['log자산총계'] = np.log(df['자산(*)(IFRS연결)(천원)'])
df['PER'] = df['PER(최고)(IFRS)'] + df['PER(최저)(IFRS)'] /2
df['연구개발비대비매출액'] = df['연구개발비(IFRS연결)(천원)'] / df['매출액(수익)(*)(IFRS연결)(천원)']

In [370]:
df = df[['회사명', '거래소코드', '부가가치율(IFRS연결)', '자기자본구성비율(IFRS연결)', '총자산대비영업현금흐름', '자기자본순이익률(IFRS연결)', '총자본순이익률(IFRS연결)', '총자본회전률(IFRS연결)', 'PBR', '이윤분배율(IFRS연결)', '자기자본증가율(IFRS연결)', '차입금의존도(IFRS연결)', 'log자산총계', 'PER', '연구개발비대비매출액', '유동자산회전률(IFRS연결)']]

In [371]:
# df = df.dropna(axis=0)

In [372]:
df['거래소코드'] = df['거래소코드'].astype(str).str.zfill(6)

In [373]:
# df상장.isna().sum()

In [374]:
df상장.dropna(axis=0, inplace=True)

In [375]:
df상장['Stock'] = df상장['Stock'].astype(int)

In [376]:
df상장['Stock'] = df상장['Stock'].astype(str).str.zfill(6)

In [377]:
df상장.columns = ['KIS', '거래소코드', '회사명_1', '신용평점']

In [378]:
df_total = pd.merge(df,df상장,how='inner',on='거래소코드')

In [379]:
df_total['신용평점']=df_total['신용평점'].map({
    'AAA' : 'AA',
    'AA':'AA',
    'A':'A',
    'BBB':'BBB',
    'BB':'BB',
    'B':'B',
    'CCC':'CCC',
    'CC':'CC',
    'C': 'D',
    'D':'D'

})

In [380]:
df_total.isna().sum()

회사명                 0
거래소코드               0
부가가치율(IFRS연결)       7
자기자본구성비율(IFRS연결)    7
총자산대비영업현금흐름         3
자기자본순이익률(IFRS연결)    7
총자본순이익률(IFRS연결)     7
총자본회전률(IFRS연결)      7
PBR                 7
이윤분배율(IFRS연결)       7
자기자본증가율(IFRS연결)     7
차입금의존도(IFRS연결)      7
log자산총계             3
PER                 7
연구개발비대비매출액          4
유동자산회전률(IFRS연결)     7
KIS                 0
회사명_1               0
신용평점                0
dtype: int64

In [381]:
df_total=df_total.dropna()

In [382]:
# df_total = df_total[(df_total != 0).all(axis=1)]

In [383]:
df_total = df_total.replace([np.inf, -np.inf], np.nan)  # inf 값을 NaN으로 대체
df_total = df_total.dropna()  # NaN이 있는 행 삭제

In [384]:
# df_total = df_total[~(df_total['부가가치율']==-20814.490000)]


In [385]:
df_total.describe()

,부가가치율(IFRS연결),자기자본구성비율(IFRS연결),총자산대비영업현금흐름,자기자본순이익률(IFRS연결),총자본순이익률(IFRS연결),총자본회전률(IFRS연결),PBR,이윤분배율(IFRS연결),자기자본증가율(IFRS연결),차입금의존도(IFRS연결),log자산총계,PER,연구개발비대비매출액,유동자산회전률(IFRS연결)
count,2227.000000,2227.000000,2227.000000,2227.000000,2227.000000,2227.000000,2227.000000,2227.000000,2227.000000,2227.000000,2227.000000,2227.000000,2227.000000,2227.000000
mean,-18.366161,58.958096,0.018190,-3.629600,-0.162317,0.824378,4.835846,-21.949250,15.421423,21.216727,26.270701,66.610238,0.328918,1.723053
std,582.062478,20.600473,0.125568,52.901335,14.605982,0.538563,10.441767,344.975082,124.259074,17.545005,1.534311,371.879712,4.512151,1.176559
min,-20814.490000,0.000000,-2.012239,-1464.690000,-121.860000,0.000000,-7.190000,-9545.350000,-1181.720000,0.000000,21.110704,-734.385000,-0.000211,0.000000
25%,10.170000,44.205000,-0.023213,-5.110000,-2.895000,0.460000,1.490000,0.000000,-1.275000,6.615000,25.216578,0.000000,0.000000,0.920000
50%,19.920000,59.610000,0.030037,4.440000,2.490000,0.730000,2.685000,14.100000,5.100000,19.150000,25.998586,16.240000,0.006658,1.550000
75%,32.600000,75.665000,0.075364,11.060000,6.575000,1.090000,5.035000,33.345000,14.420000,32.940000,27.029875,44.390000,0.035581,2.260000
max,1252.350000,98.540000,0.564594,227.300000,100.800000,4.810000,285.595000,3644.450000,3969.200000,287.450000,33.736761,9982.775000,128.164147,16.030000


In [386]:
df_total

,회사명,거래소코드,부가가치율(IFRS연결),자기자본구성비율(IFRS연결),총자산대비영업현금흐름,자기자본순이익률(IFRS연결),총자본순이익률(IFRS연결),총자본회전률(IFRS연결),PBR,이윤분배율(IFRS연결),자기자본증가율(IFRS연결),차입금의존도(IFRS연결),log자산총계,PER,연구개발비대비매출액,유동자산회전률(IFRS연결),KIS,회사명_1,신용평점
0,(주)CMG제약,058820,18.62,86.73,0.023153,-0.21,-0.19,0.40,3.980,-2.54,0.14,3.15,26.085780,0.000,0.040287,0.63,172353,(주)CMG제약,BBB
1,(주)DB하이텍,000990,63.80,77.18,0.343931,40.73,30.43,0.91,2.925,52.29,48.12,5.87,28.383710,8.195,0.045634,1.56,350354,(주)DB하이텍,A
2,(주)ES큐브,050120,49.81,90.71,-0.439465,26.29,4.10,0.12,1.745,65.89,29.87,1.71,25.524869,21.700,0.000000,0.59,013048,(주)ES큐브,A
3,(주)HJ중공업,097230,13.24,14.99,0.032938,-12.28,-2.03,0.72,2.190,-21.18,-10.88,32.74,28.574031,0.000,0.000782,1.50,492466,(주)에이치제이중공업,CCC
4,(주)MH에탄올,023150,48.81,21.32,0.036826,7.71,2.65,0.25,2.120,22.03,5.94,51.40,26.826809,22.360,0.000000,2.11,311316,(주)MH에탄올,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2233,휴림로봇(주),090710,-0.94,70.71,-0.038423,-25.80,-11.27,0.52,16.560,0.00,-21.48,13.80,25.463703,0.000,0.026830,1.47,123787,휴림로봇(주),BB
2234,휴마시스(주),205470,55.29,74.16,0.433329,66.56,45.19,1.17,4.235,70.09,71.65,0.61,26.870738,7.885,0.011108,1.41,M13779,휴마시스(주),A
2235,휴젤(주),145020,46.88,80.92,0.076803,7.31,5.89,0.27,4.080,45.96,1.93,9.59,27.679605,93.085,0.053531,0.46,388655,휴젤(주),BBB
2236,흥구석유(주),024060,5.24,85.26,0.025666,3.34,3.01,1.66,2.740,34.56,1.51,9.74,25.265865,80.395,0.000000,8.34,620378,흥구석유(주),A


In [387]:
df_total.drop('회사명_1', axis=1, inplace=True)

In [388]:
# df_total

In [389]:
df_total.columns = ['회사명', '거래소코드', '부가가치율', '자기자본구성비율', '총자산대비영업현금흐름', '자기자본순이익률', '총자본순이익률', '총자본회전률',
       'PBR', '이윤분배율', '자기자본증가율', '차입금의존도', 'log자산총계', 'PER', '연구개발비대비매출액',
       '유동자산회전률', 'KIS', '신용평점']

In [390]:
# O-Score의 계수
# coefficients = {
#     # '부가가치율' : -0.37534104,                    # 높아야 좋다
#     # '총자산대비영업현금흐름' : -0.19543408,         # 높아야 좋다 
#     '자기자본순이익률' : 0.83912683,                # 높아야 좋다
#     '총자본순이익률' : 0.22226222,                  # 높아야 좋다
#     '총자본회전률' : 0.88631003,                    # 높아야 좋다
#     'PBR' : -0.12246196,                           # 낮아야 좋다
#     '이윤분배율' : 0.11850635,                      # 높아야 좋다
#     # '자기자본증가율' : -0.37999816,                 # 높아야 좋다
#     '차입금의존도' : -0.01698886,                   # 낮아야 좋다
#     # 'PER' : -0.28375928,                           # 높아야 좋다 
#     '연구개발비대비매출액' : 0.23598311,            # 높아야 좋다
#     '유동자산회전률' : 0.05858295,                  # 높아야 좋다
#     'constant': 1.78082628
# }


coefficients = {
    '부가가치율' : -0.45957881,
    '총자산대비영업현금흐름' : -0.39609326,
    '자기자본순이익률' : 0.82401315,
    '총자본순이익률' : -0.40891006,
    '총자본회전률' : 0.90518392,
    'PBR' : -0.21364434,
    '이윤분배율' : 0.14528212,
    '자기자본증가율' : -0.53619684,
    '차입금의존도' : -0.11558369,
    'PER' : -0.22445311,
    '연구개발비대비매출액' : 0.2815131,
    '유동자산회전률' : 0.2003191,
    'constant': 0.79964512
}


# O-Score 계산
df_total['dp_score'] = coefficients['constant']
df_total['dp_score'] += coefficients['부가가치율'] * df_total['부가가치율']
df_total['dp_score'] += coefficients['총자산대비영업현금흐름'] * df_total['총자산대비영업현금흐름']
df_total['dp_score'] += coefficients['자기자본순이익률'] * df_total['자기자본순이익률']
df_total['dp_score'] += coefficients['총자본순이익률'] * df_total['총자본순이익률']
df_total['dp_score'] += coefficients['총자본회전률'] * df_total['총자본회전률']
df_total['dp_score'] += coefficients['PBR'] * df_total['PBR']
df_total['dp_score'] += coefficients['이윤분배율'] * df_total['이윤분배율']
df_total['dp_score'] += coefficients['자기자본증가율'] * df_total['자기자본증가율']
df_total['dp_score'] += coefficients['차입금의존도'] * df_total['차입금의존도']
# df_total['dp_score'] += coefficients['log자산총계'] * df_total['log자산총계']
df_total['dp_score'] += coefficients['PER'] * df_total['PER']
df_total['dp_score'] += coefficients['연구개발비대비매출액'] * df_total['연구개발비대비매출액']
df_total['dp_score'] += coefficients['유동자산회전률'] * df_total['유동자산회전률']

In [391]:
lower_bound = df_total['dp_score'].quantile(0.01)
upper_bound = df_total['dp_score'].quantile(0.99)
df_total_filtered = df_total[(df_total['dp_score'] >= lower_bound) & (df_total['dp_score'] <= upper_bound)]

In [392]:
df_total[['dp_score','신용평점']].groupby('신용평점').agg(['mean', lambda x: x.quantile(0.25), lambda x: x.quantile(0.5), lambda x: x.quantile(0.75)])

dp_score                                 
           mean <lambda_0> <lambda_1> <lambda_2>
신용평점                                            
A    -34.547959 -29.304895 -16.478281  -9.393623
AA   -24.346932 -30.248032 -20.737003 -14.226222
B    -10.717890 -42.298465 -22.741090  -9.908365
BB   -46.469291 -39.068092 -20.118387 -10.367314
BBB  -26.367369 -27.733902 -15.898386  -8.877212
CC    80.464653 -64.104019 -22.905751  17.878223
CCC  -28.966046 -57.767136 -26.348787 -11.935901
D    -34.350474 -77.580286 -38.940930  -3.042273

In [393]:
df_total[df_total['신용평점']=='CC'].isna().sum()

회사명            0
거래소코드          0
부가가치율          0
자기자본구성비율       0
총자산대비영업현금흐름    0
자기자본순이익률       0
총자본순이익률        0
총자본회전률         0
PBR            0
이윤분배율          0
자기자본증가율        0
차입금의존도         0
log자산총계        0
PER            0
연구개발비대비매출액     0
유동자산회전률        0
KIS            0
신용평점           0
dp_score       0
dtype: int64

In [394]:
df_total_filtered[['dp_score','신용평점']].groupby('신용평점').mean() + 50

,dp_score
신용평점,
A,26.336326
AA,25.653068
B,17.929821
BB,15.989016
BBB,25.119222
CC,16.629058
CCC,16.026914
D,5.077885


In [395]:
df_total

,회사명,거래소코드,부가가치율,자기자본구성비율,총자산대비영업현금흐름,자기자본순이익률,총자본순이익률,총자본회전률,PBR,이윤분배율,자기자본증가율,차입금의존도,log자산총계,PER,연구개발비대비매출액,유동자산회전률,KIS,신용평점,dp_score
0,(주)CMG제약,058820,18.62,86.73,0.023153,-0.21,-0.19,0.40,3.980,-2.54,0.14,3.15,26.085780,0.000,0.040287,0.63,172353,BBB,-9.021094
1,(주)DB하이텍,000990,63.80,77.18,0.343931,40.73,30.43,0.91,2.925,52.29,48.12,5.87,28.383710,8.195,0.045634,1.56,350354,A,-27.737497
2,(주)ES큐브,050120,49.81,90.71,-0.439465,26.29,4.10,0.12,1.745,65.89,29.87,1.71,25.524869,21.700,0.000000,0.59,013048,A,-13.588972
3,(주)HJ중공업,097230,13.24,14.99,0.032938,-12.28,-2.03,0.72,2.190,-21.18,-10.88,32.74,28.574031,0.000,0.000782,1.50,492466,CCC,-15.129933
4,(주)MH에탄올,023150,48.81,21.32,0.036826,7.71,2.65,0.25,2.120,22.03,5.94,51.40,26.826809,22.360,0.000000,2.11,311316,B,-27.125627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2233,휴림로봇(주),090710,-0.94,70.71,-0.038423,-25.80,-11.27,0.52,16.560,0.00,-21.48,13.80,25.463703,0.000,0.026830,1.47,123787,BB,-8.247034
2234,휴마시스(주),205470,55.29,74.16,0.433329,66.56,45.19,1.17,4.235,70.09,71.65,0.61,26.870738,7.885,0.011108,1.41,M13779,A,-18.050577
2235,휴젤(주),145020,46.88,80.92,0.076803,7.31,5.89,0.27,4.080,45.96,1.93,9.59,27.679605,93.085,0.053531,0.46,388655,BBB,-34.040187
2236,흥구석유(주),024060,5.24,85.26,0.025666,3.34,3.01,1.66,2.740,34.56,1.51,9.74,25.265865,80.395,0.000000,8.34,620378,A,-12.468848
